1st


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import timm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

# ==== 1. SETUP GPU ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ==== 2. PARAMETERS ====
data_dir = "/kaggle/input/augmented-pneumonia-dataset/augmented/train"
batch_size = 32
epochs = 10
learning_rate = 1e-3
num_folds = 5  # Number of folds for cross-validation

# ==== 3. DATA TRANSFORMATIONS ====
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ==== 4. LOAD DATASET ====
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
targets = np.array(dataset.targets)  # Get labels for stratified splitting
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)  # Stratified K-Fold

# ==== 5. CROSS-VALIDATION ====
best_auc = 0.0
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    print(f"\n Fold {fold + 1}/{num_folds}")

    # Subset dataset for current fold
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    # DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # ==== 6. MODEL SETUP ====
    model = timm.create_model('xception', pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 1)
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    # ==== 7. TRAINING ====
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        all_preds, all_labels = [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            all_preds.extend(torch.sigmoid(outputs).cpu().detach().numpy())
            all_labels.extend(labels.cpu().detach().numpy())

        train_auc = roc_auc_score(all_labels, all_preds)

        # ==== 8. VALIDATION ====
        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_preds.extend(torch.sigmoid(outputs).cpu().detach().numpy())
                val_labels.extend(labels.cpu().detach().numpy())

        val_auc = roc_auc_score(val_labels, val_preds)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss/len(train_loader):.4f} - Train AUC: {train_auc:.4f} - Val Loss: {val_loss/len(val_loader):.4f} - Val AUC: {val_auc:.4f}")
        
        scheduler.step()
        
    fold_results.append(best_auc)

# ==== 9. FINAL CROSS-VALIDATION RESULTS ====
print("\n Cross-validation completed")
print(f"Average AUC: {np.mean(fold_results):.4f} ± {np.std(fold_results):.4f}")


Using device: cuda

 Fold 1/5


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


Epoch 1/10 - Train Loss: 0.1214 - Train AUC: 0.9882 - Val Loss: 0.1122 - Val AUC: 0.9949
Epoch 2/10 - Train Loss: 0.0733 - Train AUC: 0.9954 - Val Loss: 0.0611 - Val AUC: 0.9969
Epoch 3/10 - Train Loss: 0.0526 - Train AUC: 0.9976 - Val Loss: 0.0661 - Val AUC: 0.9972
Epoch 4/10 - Train Loss: 0.0148 - Train AUC: 0.9999 - Val Loss: 0.0342 - Val AUC: 0.9992
Epoch 5/10 - Train Loss: 0.0051 - Train AUC: 1.0000 - Val Loss: 0.0335 - Val AUC: 0.9992
Epoch 6/10 - Train Loss: 0.0022 - Train AUC: 1.0000 - Val Loss: 0.0305 - Val AUC: 0.9994
Epoch 7/10 - Train Loss: 0.0012 - Train AUC: 1.0000 - Val Loss: 0.0337 - Val AUC: 0.9994
Epoch 8/10 - Train Loss: 0.0009 - Train AUC: 1.0000 - Val Loss: 0.0309 - Val AUC: 0.9994
Epoch 9/10 - Train Loss: 0.0008 - Train AUC: 1.0000 - Val Loss: 0.0335 - Val AUC: 0.9994
Epoch 10/10 - Train Loss: 0.0006 - Train AUC: 1.0000 - Val Loss: 0.0314 - Val AUC: 0.9994

 Fold 2/5


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Epoch 1/10 - Train Loss: 0.1176 - Train AUC: 0.9887 - Val Loss: 0.1035 - Val AUC: 0.9947
Epoch 2/10 - Train Loss: 0.0729 - Train AUC: 0.9955 - Val Loss: 0.0837 - Val AUC: 0.9964
Epoch 3/10 - Train Loss: 0.0559 - Train AUC: 0.9973 - Val Loss: 0.0551 - Val AUC: 0.9976
Epoch 4/10 - Train Loss: 0.0188 - Train AUC: 0.9997 - Val Loss: 0.0312 - Val AUC: 0.9992
Epoch 5/10 - Train Loss: 0.0070 - Train AUC: 1.0000 - Val Loss: 0.0325 - Val AUC: 0.9993
Epoch 6/10 - Train Loss: 0.0046 - Train AUC: 1.0000 - Val Loss: 0.0351 - Val AUC: 0.9993
Epoch 7/10 - Train Loss: 0.0025 - Train AUC: 1.0000 - Val Loss: 0.0337 - Val AUC: 0.9994
Epoch 8/10 - Train Loss: 0.0020 - Train AUC: 1.0000 - Val Loss: 0.0310 - Val AUC: 0.9994
Epoch 9/10 - Train Loss: 0.0021 - Train AUC: 1.0000 - Val Loss: 0.0314 - Val AUC: 0.9995
Epoch 10/10 - Train Loss: 0.0016 - Train AUC: 1.0000 - Val Loss: 0.0314 - Val AUC: 0.9995

 Fold 3/5


...

Epoch 1/10 - Train Loss: 0.1202 - Train AUC: 0.9881 - Val Loss: 0.2149 - Val AUC: 0.9816
Epoch 2/10 - Train Loss: 0.0673 - Train AUC: 0.9961 - Val Loss: 0.1010 - Val AUC: 0.9957
Epoch 3/10 - Train Loss: 0.0528 - Train AUC: 0.9976 - Val Loss: 0.0677 - Val AUC: 0.9978
Epoch 4/10 - Train Loss: 0.0111 - Train AUC: 0.9999 - Val Loss: 0.0293 - Val AUC: 0.9994
Epoch 5/10 - Train Loss: 0.0037 - Train AUC: 1.0000 - Val Loss: 0.0319 - Val AUC: 0.9993
Epoch 6/10 - Train Loss: 0.0037 - Train AUC: 0.9999 - Val Loss: 0.0337 - Val AUC: 0.9994
Epoch 7/10 - Train Loss: 0.0015 - Train AUC: 1.0000 - Val Loss: 0.0272 - Val AUC: 0.9993
Epoch 8/10 - Train Loss: 0.0013 - Train AUC: 1.0000 - Val Loss: 0.0283 - Val AUC: 0.9994
Epoch 9/10 - Train Loss: 0.0008 - Train AUC: 1.0000 - Val Loss: 0.0284 - Val AUC: 0.9994
Epoch 10/10 - Train Loss: 0.0007 - Train AUC: 1.0000 - Val Loss: 0.0274 - Val AUC: 0.9994

 Fold 4/5


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Epoch 1/10 - Train Loss: 0.1214 - Train AUC: 0.9880 - Val Loss: 0.0798 - Val AUC: 0.9960
Epoch 2/10 - Train Loss: 0.0729 - Train AUC: 0.9956 - Val Loss: 0.0613 - Val AUC: 0.9979
Epoch 3/10 - Train Loss: 0.0518 - Train AUC: 0.9976 - Val Loss: 0.0552 - Val AUC: 0.9979
Epoch 4/10 - Train Loss: 0.0148 - Train AUC: 0.9998 - Val Loss: 0.0332 - Val AUC: 0.9990
Epoch 5/10 - Train Loss: 0.0053 - Train AUC: 1.0000 - Val Loss: 0.0324 - Val AUC: 0.9992
Epoch 6/10 - Train Loss: 0.0023 - Train AUC: 1.0000 - Val Loss: 0.0312 - Val AUC: 0.9992
Epoch 7/10 - Train Loss: 0.0012 - Train AUC: 1.0000 - Val Loss: 0.0298 - Val AUC: 0.9993
Epoch 8/10 - Train Loss: 0.0012 - Train AUC: 1.0000 - Val Loss: 0.0288 - Val AUC: 0.9993
Epoch 9/10 - Train Loss: 0.0009 - Train AUC: 1.0000 - Val Loss: 0.0291 - Val AUC: 0.9993
Epoch 10/10 - Train Loss: 0.0006 - Train AUC: 1.0000 - Val Loss: 0.0291 - Val AUC: 0.9993

 Fold 5/5


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Epoch 1/10 - Train Loss: 0.1210 - Train AUC: 0.9879 - Val Loss: 0.0650 - Val AUC: 0.9967
Epoch 2/10 - Train Loss: 0.0770 - Train AUC: 0.9951 - Val Loss: 0.1122 - Val AUC: 0.9967
Epoch 3/10 - Train Loss: 0.0555 - Train AUC: 0.9973 - Val Loss: 0.0392 - Val AUC: 0.9988
Epoch 4/10 - Train Loss: 0.0147 - Train AUC: 0.9999 - Val Loss: 0.0246 - Val AUC: 0.9995
Epoch 5/10 - Train Loss: 0.0051 - Train AUC: 1.0000 - Val Loss: 0.0237 - Val AUC: 0.9996
Epoch 6/10 - Train Loss: 0.0026 - Train AUC: 1.0000 - Val Loss: 0.0233 - Val AUC: 0.9996
Epoch 7/10 - Train Loss: 0.0010 - Train AUC: 1.0000 - Val Loss: 0.0237 - Val AUC: 0.9997
Epoch 8/10 - Train Loss: 0.0008 - Train AUC: 1.0000 - Val Loss: 0.0230 - Val AUC: 0.9996
Epoch 9/10 - Train Loss: 0.0008 - Train AUC: 1.0000 - Val Loss: 0.0237 - Val AUC: 0.9996
Epoch 10/10 - Train Loss: 0.0006 - Train AUC: 1.0000 - Val Loss: 0.0234 - Val AUC: 0.9997

 Cross-validation completed
Average AUC: 0.0000 ± 0.0000


<a href="/kaggle/working/xception-imagenet-aug-6.csv"> Download File </a>